### Imports

In [139]:
from simcse import SimCSE
import pandas as pd
import numpy as np
from pathlib import Path

### Load model and dataset

In [6]:
model = SimCSE('./data/dental_model/')

Some weights of BertModel were not initialized from the model checkpoint at ./data/dental_model/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
dataset = pd.read_csv('data/input/sts_simcse_test_input.csv')

### Process

In [74]:
sentence1_values = [str(i) for i in list(dataset['sentence1'].values)]
sentence2_values = [str(i) for i in list(dataset['sentence2'].values)]

In [73]:
sentence1_embedding = model.encode(sentence1_values)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 31.26it/s]


In [80]:
sentence2_embedding = model.encode(sentence2_values)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 30.79it/s]


In [106]:
len(sentence2_embedding[0])

768

In [131]:
sts_clutser_embeddings = []

# Loop both embeddings and enter them in the new array
for i in range(dataset.shape[0]):
    sts_clutser_embeddings.append([i,1,np.array(sentence1_embedding[i])])
    sts_clutser_embeddings.append([i,2,np.array(sentence2_embedding[i])])

In [135]:
sts_cluster_input = pd.DataFrame(sts_clutser_embeddings)
sts_cluster_input.columns = ['row','sentence number','sentence embedding']

In [137]:
sts_cluster_input

,row,sentence number,sentence embedding
0,0,1,"[0.018946992, -0.033017647, 0.0584192, 0.03188..."
1,0,2,"[0.022349114, -0.019896414, 0.068644755, 0.016..."
2,1,1,"[-0.0063565667, 0.038259946, 0.025764938, 0.00..."
3,1,2,"[0.00076834724, 0.009331502, 0.037513718, 0.02..."
4,2,1,"[0.01108482, -0.006460412, 0.05317341, 0.02150..."
...,...,...,...
2753,1376,2,"[0.030478576, 0.024279015, 0.060241528, -0.030..."
2754,1377,1,"[-0.080806784, 0.019064022, 0.029170275, -0.02..."
2755,1377,2,"[-0.014855488, 0.072309084, 0.027893208, -0.04..."
2756,1378,1,"[0.066184744, -0.042154677, -0.013624112, 0.04..."


In [141]:
# Check folder exists
Path('./data/output').mkdir(parents=True, exist_ok=True)

# Export pandas dataframe to .csv
sts_cluster_input.to_csv('data/output/sts_cluster_input.csv')